This code produces Figure 9 in the manuscript: compariosn of nonlinear solutions with the steady Green's function solution. 

The code requires FEniCSx---see the README for details.

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, './source')

In [ ]:
from main import solve
from params import H, Nx, Nz, rho_w, rho_i, t_e
from smb import smb_h, smb_s
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve
import numpy as np
from params import t_r,t_f,nt,L
from scipy.interpolate import griddata,interp1d
from scipy.optimize import curve_fit

In [ ]:
m0 = 5 / 3.154e7                # max basal melt(+) or freeze(-) rate (m/yr)
stdev = 10*H/3                  # standard deviation for Gaussian basal melt anomaly

Solve problem with small melt rate:

In [ ]:
a = lambda x,t: smb_h(x,t,0.2*m0,stdev)
m = lambda x,t: smb_s(x,t,0.2*m0,stdev)

h,s,u,w,x,x_vel,z_vel = solve(a,m)

In [ ]:
t = np.linspace(0,t_f, nt)

In [ ]:
H_max = np.max(np.abs(h-s - H),axis=0)

In [ ]:
H_decay = H_max[t>8*t_e]
t_decay = t[t>8*t_e]
t_0 = t_decay[0]
t_decay -= t_0

funct = lambda x, a, b, c: a * np.exp(-b * x) + c

y_data = H_decay/H
x_data = t_decay/t_e

popt, pcov = curve_fit(func, x_data, y_data)

a,b,c = popt

In [ ]:
plt.figure(figsize=(8,6))
plt.title(r'$t_\mathrm{decay}=$'+'{:.2f}'.format(1/b)+r' $t_\mathrm{e}$' ,fontsize=20)
plt.plot(t/t_e,H_max/H,color='slateblue',linewidth=3)
plt.plot(x_data+t_0/t_e,func(x_data, a, b, c),color='k',linestyle='--',linewidth=3,label=r'exponential fit')
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel(r'$t\,/\,t_\mathrm{e}$',fontsize=20)
plt.ylabel(r'max $|H\,/\,H_0-1| $',fontsize=20)
plt.axvline(x=8,linestyle='--',color='crimson',linewidth=2)
plt.annotate(xy=(6.0,0.0),text=r'turn off melt', fontsize=16,color='crimson',bbox=dict(facecolor='w', edgecolor='k', boxstyle='round'))
plt.legend(fontsize=12)
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(x_data,y_data,color='k',linewidth=3)
plt.plot(x_data,func(x_data, a, b, c),color='crimson',linestyle='--',linewidth=3)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
# plt.yscale('log')
# plt.xscale('log')
plt.xlabel(r'$t\,/\,t_\mathrm{e}$',fontsize=20)
plt.ylabel(r'max $|H\,/\,H_0-1| $',fontsize=20)
plt.show()
plt.close()